## Build a pipeline for real time detection 

In [1]:
from roboflow import Roboflow
from transformers import DetrImageProcessor
import os
import torchvision
from PIL import Image
import random
import cv2
import numpy as np
import supervision as sv
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import pytorch_lightning as pl
from transformers import DetrForObjectDetection
import torch

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
image_processor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')

In [4]:
box_annot = sv.BoxAnnotator()

In [5]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model1 = DetrForObjectDetection.from_pretrained('/Users/sudipkhadka/Desktop/Object_Detection/Research/trained_model')
model1.to(DEVICE)

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [6]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Error: Could not access camera')
    exit()
print("Searching for License Plates........ Press 'q' to exit")
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Error: Could not read frame')
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        with torch.no_grad():
            input = image_processor(images=img, return_tensors='pt').to(DEVICE)
            outputs = model1(**input)

            CONFIDENCE_THRESHOLD =  0.5
            target_size = torch.tensor([img.shape[:2]]).to(DEVICE)
            results = image_processor.post_process_object_detection(outputs=outputs, threshold=CONFIDENCE_THRESHOLD, target_sizes=target_size)[0]

            detections = sv.Detections.from_transformers(transformers_results=results)
            frame = box_annot.annotate(scene=img.copy(), detections=detections) 

            cv2.imshow('License Plate Detection', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break   
finally:
    cap.release()
    cv2.destroyAllWindows()
    print('Camera Released and all windows closed')

Searching for License Plates........ Press 'q' to exit


2025-01-29 19:27:19.955 python[2393:68984] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-29 19:27:19.955 python[2393:68984] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Camera Released and all windows closed
